# Lecture 10: Power Analysis
A power analysis is a process for deciding what sample size to use based on the chance of observing the minimum effect you are looking for in your study. This power analysis uses [DeclareDesign](http://declaredesign.org/). Another option is the [egap Power Analysis page.](https://egap.org/content/power-analysis-simulations-r)

In the following example, we:
* establish *assumptions* about the experiment that we can *simulate*
* simulate the experiment N times
* *diagnose* the experiment design in light of our assumptions, for characteristics including:
  * statistical power (chance of observing a statistically-significant result)
  * bias in the results
  * (more below)

In [ ]:
options("scipen"=9, "digits"=4)
library(dplyr)
library(MASS)
library(ggplot2)
library(rlang)
library(corrplot)
library(Hmisc)
library(tidyverse)
library(viridis)
library(fabricatr)
library(DeclareDesign)
## Installed DeclareDesign 0.13 using the following command:
# install.packages("DeclareDesign", dependencies = TRUE,
#                 repos = c("http://R.declaredesign.org", "https://cloud.r-project.org"))
options(repr.plot.width=7, repr.plot.height=4)

set.seed(03456920)

sessionInfo()

# DeclareDesign Example
This example defines a simple difference in means experiment with two arms.

In [ ]:
config.df <- data.frame(
    pa.label = "example.design",
    n.max = 200,
    n.min = 10,
    
    y.mean.a    = 0,
    y.sd.both   = 1,
    y.effect.b  = 1
)

### Declare Population and Potential Outcomes
The first step is to declare the population and potential outcomes:

In [ ]:
design <-
    declare_population(
        N = config.df$n.min
    ) +
    declare_potential_outcomes(
        YA_Z_0 = rnorm(n=N, mean = config.df$y.mean.a, sd = config.df$y.sd.both),
        YA_Z_1 = rnorm(n=N, mean = config.df$y.mean.a + config.df$y.effect.b, sd = config.df$y.sd.both )
    )
design

### Declare Assignment
The next step is to declare the assignment/randomization approach

In [ ]:
design <- design +
    declare_assignment(num_arms = 2,
                       conditions = (c("0", "1")))
design

### Declare the Estimand
The next step is to declare the estimand, the "true effect" that the experiment is estimating

In [ ]:
design <- design + 
    declare_estimand(ate_YA_1_0 = config.df$y.effect.b)
design

### Declare the "Reveal"
Describe the process of assigning conditions and observing variables (this code can get complex)

In [ ]:
design <- design +
    declare_reveal(outcome_variables = c("YA"))
design

### Declare the Estimator
Next, declare the estimator, the means by which you attempt to estimate the estimand.

In [ ]:
design <- design +
    declare_estimator(YA ~ Z, estimand="ate_YA_1_0")
design

### Diagnose the Design
To diagnose the design, you can simulate the research design and observe characteristics of the total research design in relation to your assumptions.

#### "Diagnosands" (results of the diagnosis)
* **Bias** Difference between the mean *estimate* and the *estimand*
* **RMSE** Root mean square error of the estimate sqrt(mean((estimate - estimand) ^ 2))
* **Coverage** Probability of confidence intervals including the *estimand*
* **Mean Estimate** The mean of all simulated *estimates*
* **SD Estimate** The standard deviation of all simulated *estimates*
* **Type S Rate** Probability of experiment to produce a statistically-significant result that is opposite in sign from the estimand
* **Mean Estimand** The mean of the *estimand*

In [ ]:
diagnose_design(design, sims=500, bootstrap_sims=500)

# Conduct a Power Analysis
## Utility Methods

In [ ]:
# Return the minimum power reported in a diagnosis
# 
#` @param diagnosis
min.diagnosis.power <- function(diagnosis){
    min(diagnosis$diagnosands_df['power'])
}


# CONSTRUCT A DIAGNOSIS
#
#' @param n.size     the sample size to test
#' @param config.df  the configuration dataframe to use 
#' @param sims.count the number of simulations to carry out
#' @param bootstrap.sims.count the number of bootstraps for estimating the simulation test statistics  

diagnose.experiment <- function(n.size, config.df, sims.count=500, bootstrap.sims.count=500){
    
    declare_population(
        N = n.size
    ) +
    declare_potential_outcomes(
        YA_Z_0 = rnorm(n=N, mean = config.df$y.mean.a, sd = config.df$y.sd.both),
        YA_Z_1 = rnorm(n=N, mean = config.df$y.mean.a + config.df$y.effect.b, sd = config.df$y.sd.both )
    ) +
    declare_assignment(num_arms = 2,
                       conditions = (c("0", "1"))) +
    declare_estimand(ate_YA_1_0 = effect.b) +
    declare_reveal(outcome_variables = c("YA")) +
    declare_estimator(YA ~ Z, estimand="ate_YA_1_0")
    
    diagnosis <- diagnose_design(design, sims = sims.count, 
                                 bootstrap_sims = bootstrap.sims.count)
    diagnosis
}

In [ ]:
# Iterate linearly for a certain level of statistical power
# within the constraints of a configuration file
# at a certain sample size increment. Useful for
# illustrating ideas, or for comparing estimators with
# very different statistical power, where the binary search
# will optimize for the worst estimator but not show useful
# indormation about more efficient estimators
#
#` @param config.df The configuration file in question
#` @diagnosis.method The method that conducts a single DeclareDesign diagnosis and returns the diagnosis
#` @iteration.interval when iterating, use this interval between sample sizes

iterate.for.power <- function(config.df, diagnosis.method = diagnose.experiment, 
                             iteration.interval){  
    max.sample.size = config.df$n.max
    min.sample.size = config.df$n.min
    current.sample.size = min.sample.size
    
    iteration.count = ceiling((max.sample.size - min.sample.size) / iteration.interval)

    ## Initialize first iteration
    print(paste("min:", min.sample.size, "max:", max.sample.size, "current:", current.sample.size))
    flush.console()

    ptm = proc.time()
    ddf <- diagnosis.method(current.sample.size, config.df)
    ddf$diagnosands$n <- current.sample.size
    diagnoses.df = ddf$diagnosands
    current.power <- min.diagnosis.power(ddf)
    time.elapsed <- proc.time() -  ptm
    print(paste("     seconds:", as.integer(time.elapsed['elapsed'])))
    
    for(i in seq(1, iteration.count)){
        current.sample.size = current.sample.size + iteration.interval
        print(paste("min:", min.sample.size, "max:", max.sample.size, "current:", current.sample.size))
        flush.console()
    
        ptm = proc.time()
        ## conduct simulations
        ddf <- diagnosis.method(current.sample.size, config.df)
        ddf$diagnosands$n <- current.sample.size
        ## append simulation results to dataframe
        diagnoses.df <- rbind(diagnoses.df, ddf$diagnosands)
        time.elapsed <- proc.time() -  ptm
        print(paste("     seconds:", as.integer(time.elapsed['elapsed'])))
    }
    diagnoses.df
}

In [ ]:
# Create a plot of a power search or iteration output
# Especially useful in cases with multiple DVs or models
#
#' @param diagnoses Dataframe of diagnosis info
#` @param config.df the power analysis config dataframe

plot.power.results <- function(diagnoses, config.df){
    for(estimator_label in unique(diagnoses$estimator_label)){
        estimator.diagnoses <- diagnoses[diagnoses$estimator_label==estimator_label,]
        estimator_min_sample = min(estimator.diagnoses$n[estimator.diagnoses$power>0.8])
        p <- ggplot(data=estimator.diagnoses, aes(n, power)) +
                geom_point(color="coral") +
                xlab("sample size") +
                ylim(0,1) +
                geom_hline(aes(yintercept=0.8), linetype="dashed") +
                theme_light() +
                ggtitle(paste(config.df$pa.label, ": statistical Power for Estimator ", estimator_label, "\n",
                              "Minimum sample: ", estimator_min_sample, sep="")) +
                ggsave(paste("figures/power.analysis.", make.names(estimator_label), ".", config.df$pa.label, ".png", sep=""))
    }
}



# Conduct Power Analysis
In this code, we simulate the experiment at different sample sizes and plot the diagnosands by sample size.

In [ ]:
interval = 10
power.iterate.df <- iterate.for.power(config.df, iteration.interval = interval)

### Plot Statistical Power

In [ ]:
ggplot(power.iterate.df, aes(n, power, color=estimator_label)) +
    ## CHART SUBSTANCE
    geom_line() +
    geom_point() +
    ## LABELS AND COSMETICS
    geom_hline(yintercept=0.8, size=0.25) +
    theme_bw(base_size = 12, base_family = "Helvetica") +
    theme(axis.text.x = element_text(angle=45, hjust = 1)) +
    scale_y_continuous(breaks = seq(0,1,0.1), limits = c(0,1), labels=scales::percent) +
    scale_x_continuous(breaks = seq(config.df$n.min,config.df$n.max+10,interval)) +
    scale_color_viridis(discrete=TRUE) +
    xlab("sample size") +
    ggtitle("Statistical Power Associated with Estimators")

### Plot Estimator Bias

In [ ]:
ggplot(power.iterate.df, aes(n, bias, color=estimator_label)) +
    ## CHART SUBSTANCE
    geom_line() +
    geom_jitter(width=20, height=0) +
    ## LABELS AND COSMETICS
    theme_bw(base_size = 12, base_family = "Helvetica") +
    theme(axis.text.x = element_text(angle=45, hjust = 1)) +
    scale_x_continuous(breaks = seq(config.df$n.min,config.df$n.max+10,interval)) +
    scale_color_viridis(discrete=TRUE) +
    xlab("sample size") +
    ggtitle("Mean Bias Associated with Estimators")

### Plot Confidence Interval Coverage

In [ ]:
ggplot(power.iterate.df, aes(n, coverage, color=estimator_label)) +
    ## CHART SUBSTANCE
    geom_line() +
    geom_point() +
    ## LABELS AND COSMETICS
    theme_bw(base_size = 12, base_family = "Helvetica") +
    theme(axis.text.x = element_text(angle=45, hjust = 1)) +
    scale_x_continuous(breaks = seq(config.df$n.min,config.df$n.max+10,interval)) +
    scale_y_continuous(breaks = seq(0,1,0.1), limits = c(0,1), labels=scales::percent) +
    scale_color_viridis(discrete=TRUE) +
    xlab("sample size") +
    ggtitle("Mean Confidence Interval Coverage Associated with Estimators")

### Plot Type S Rate

In [ ]:
ggplot(power.iterate.df, aes(n, type_s_rate, color=estimator_label)) +
    ## CHART SUBSTANCE
    geom_line() +
    geom_point() +
    ## LABELS AND COSMETICS
    theme_bw(base_size = 12, base_family = "Helvetica") +
    theme(axis.text.x = element_text(angle=45, hjust = 1)) +
    scale_x_continuous(breaks = seq(config.df$n.min,config.df$n.max+10,interval)) +
    scale_y_continuous(breaks = seq(0,0.2,0.02), limits = c(0,0.2), labels=scales::percent) +
    scale_color_viridis(discrete=TRUE) +
    xlab("sample size") +
    ggtitle("Incorrect Sign of Estimate Associated with Estimators")